# Test Concepts Processing - Standalone Test

**Purpose:** Test the test concepts extraction logic with a specific document.

**Algorithm:**
1. Query transfer variables from silver table for the given `parent_document_id`
2. Load the Excel file from `md_file_history`
3. Find test_concepts sheets from `md_dta_excel_file_raw`
4. Detect header row by matching against known transfer variable names
5. Create `transfer_tuple_map` for each data row
6. Compute `definition_hash` from the tuple map

**Output Table:** `silver_md.md_dta_vendor_test_concepts_draft`


In [ ]:
# Cell 0: Install Excel dependencies
%pip install openpyxl xlrd --quiet

# Note: If imports fail after this, run: dbutils.library.restartPython()


In [ ]:
# Cell 1: Configuration
import base64
import hashlib
import json
import re
import uuid
from io import BytesIO
from datetime import datetime

import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    BooleanType, ArrayType, MapType, TimestampType
)

print("="*80)
print("TEST CONCEPTS PROCESSING - STANDALONE TEST")
print("="*80)


In [ ]:
# Cell 2: Parameters
# Set these for testing

CATALOG = "aira_test"
BRONZE_SCHEMA = "bronze_md"
SILVER_SCHEMA = "silver_md"

# Get parent_document_id from widget or hardcode for testing
try:
    PARENT_DOCUMENT_ID = dbutils.widgets.get("parent_document_id")
except:
    # Hardcode for testing - replace with an actual document ID from your data
    PARENT_DOCUMENT_ID = None

# If no document ID provided, get the latest one from history
if not PARENT_DOCUMENT_ID:
    print("No parent_document_id provided, fetching latest from history table...")
    latest_doc = spark.sql(f"""
        SELECT document_id 
        FROM {CATALOG}.{BRONZE_SCHEMA}.md_file_history 
        WHERE is_current = true 
        ORDER BY created_ts DESC 
        LIMIT 1
    """).first()
    if latest_doc:
        PARENT_DOCUMENT_ID = latest_doc['document_id']
    else:
        raise ValueError("No documents found in history table")

print(f"Catalog: {CATALOG}")
print(f"Parent Document ID: {PARENT_DOCUMENT_ID}")


In [ ]:
# Cell 3: Table Names
history_table = f"{CATALOG}.{BRONZE_SCHEMA}.md_file_history"
sheets_table = f"{CATALOG}.{BRONZE_SCHEMA}.md_dta_excel_file_raw"
transfer_vars_table = f"{CATALOG}.{SILVER_SCHEMA}.md_dta_transfer_variables_draft"
output_table = f"{CATALOG}.{SILVER_SCHEMA}.md_dta_vendor_test_concepts_draft"

print(f"History table: {history_table}")
print(f"Sheets table: {sheets_table}")
print(f"Transfer variables table: {transfer_vars_table}")
print(f"Output table: {output_table}")


In [ ]:
# Cell 4: Load Transfer Variables for this Document
# These are the known column names we'll look for in test concepts sheets

print(f"\n{'='*60}")
print("Loading transfer variables for this document...")
print(f"{'='*60}")

df_transfer_vars = spark.table(transfer_vars_table).filter(
    F.col("parent_document_id") == PARENT_DOCUMENT_ID
).select("transfer_variable_name", "trial_id", "data_stream_type", "data_provider_name")

transfer_vars_count = df_transfer_vars.count()
print(f"Found {transfer_vars_count} transfer variables")

if transfer_vars_count == 0:
    raise ValueError(f"No transfer variables found for parent_document_id={PARENT_DOCUMENT_ID}")

# Get unique transfer variable names
transfer_vars_rows = df_transfer_vars.select("transfer_variable_name").distinct().collect()
transfer_variable_names = {row['transfer_variable_name'].upper() for row in transfer_vars_rows}
print(f"Unique transfer variable names: {len(transfer_variable_names)}")
print(f"Sample: {list(transfer_variable_names)[:10]}...")

# Get metadata
metadata_row = df_transfer_vars.select(
    "trial_id", "data_stream_type", "data_provider_name"
).first()

TRIAL_ID = metadata_row['trial_id']
DATA_STREAM_TYPE = metadata_row['data_stream_type']
DATA_PROVIDER_NAME = metadata_row['data_provider_name']

print(f"Trial ID: {TRIAL_ID}")
print(f"Data Stream Type: {DATA_STREAM_TYPE}")
print(f"Data Provider: {DATA_PROVIDER_NAME}")


In [ ]:
# Cell 5: Find Test Concepts Sheets
print(f"\n{'='*60}")
print("Finding test_concepts sheets...")
print(f"{'='*60}")

df_sheets = spark.table(sheets_table).filter(
    (F.col("parent_document_id") == PARENT_DOCUMENT_ID) &
    (F.col("sheet_category") == "test_concepts")
).select("document_id", "sheet_name", "source_file_path")

sheets_count = df_sheets.count()
print(f"Found {sheets_count} test_concepts sheet(s)")

if sheets_count == 0:
    print("\n⚠️ No test_concepts sheets found for this document.")
    print("Check if the sheet_category was correctly assigned during Excel extraction.")
    
    # Show all sheets for debugging
    print("\nAll sheets for this document:")
    spark.table(sheets_table).filter(
        F.col("parent_document_id") == PARENT_DOCUMENT_ID
    ).select("sheet_name", "sheet_category").show(truncate=False)
else:
    df_sheets.show(truncate=False)


In [ ]:
# Cell 6: Load Excel Content
print(f"\n{'='*60}")
print("Loading Excel content from history table...")
print(f"{'='*60}")

df_content = spark.table(history_table).filter(
    (F.col("document_id") == PARENT_DOCUMENT_ID) &
    (F.col("is_current") == True)
).select("content_base64").first()

if not df_content:
    raise ValueError(f"Document not found in history: {PARENT_DOCUMENT_ID}")

excel_bytes = base64.b64decode(df_content['content_base64'])
print(f"Excel content loaded: {len(excel_bytes):,} bytes")


In [ ]:
# Cell 7: Define Test Concepts Processing Functions

def normalize_header(val) -> str:
    """Normalize a header value to uppercase with underscores."""
    if pd.isna(val) or val is None:
        return None
    h = str(val).strip().upper()
    if not h or h.lower() in ('nan', 'none', 'unnamed'):
        return None
    h = re.sub(r'\s+', '_', h)
    h = re.sub(r'[^A-Z0-9_]', '', h)
    return h if h else None


def find_header_row_by_transfer_vars(
    excel_bytes: bytes,
    sheet_name: str,
    transfer_variable_names: set,
    max_scan_rows: int = 25
) -> tuple:
    """
    Find the header row by matching against known transfer variable names.
    Fixed columns (Tab Name, Test Concept Reference) are detected by POSITION
    (columns before the first transfer variable column).
    
    Args:
        excel_bytes: Raw Excel file content
        sheet_name: Sheet to process
        transfer_variable_names: Set of known transfer variable names (uppercase)
        max_scan_rows: How many rows to scan for headers
    
    Returns:
        (header_row_index, col_idx_to_name_map, fixed_columns_map)
    """
    raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, dtype=str)
    
    best_row = None
    best_score = 0
    best_tv_map = {}  # col_idx -> transfer_variable_name
    
    for row_idx in range(min(max_scan_rows, len(raw_df))):
        row_values = raw_df.iloc[row_idx].values
        tv_map = {}
        
        for col_idx, val in enumerate(row_values):
            normalized = normalize_header(val)
            if normalized and normalized in transfer_variable_names:
                tv_map[col_idx] = normalized
        
        # Score based on number of matched transfer variables
        if len(tv_map) > best_score:
            best_score = len(tv_map)
            best_row = row_idx
            best_tv_map = tv_map
    
    if best_row is None or best_score < 3:
        print(f"  ⚠️ Could not find header row with transfer variables (best score: {best_score})")
        return None, {}, {}
    
    print(f"  ✓ Header row found at index {best_row} with {best_score} transfer variables")
    
    # Find first transfer variable column index
    first_tv_col = min(best_tv_map.keys()) if best_tv_map else 0
    
    # Fixed columns are those BEFORE the first transfer variable column
    # Based on typical structure: Column 0 = Tab Name, Column 1 = Test Concept Reference
    fixed_columns = {}
    if first_tv_col >= 1:
        fixed_columns[0] = 'TAB_NAME'
    if first_tv_col >= 2:
        fixed_columns[1] = 'TEST_CONCEPT_REFERENCE'
    
    print(f"  ✓ Fixed columns detected by position: {list(fixed_columns.values())} (before column {first_tv_col})")
    
    return best_row, best_tv_map, fixed_columns


def compute_definition_hash(transfer_tuple_map: dict) -> str:
    """Compute SHA256 hash from the transfer_tuple_map."""
    if not transfer_tuple_map:
        return None
    sorted_items = sorted(transfer_tuple_map.items())
    canonical = json.dumps(sorted_items, ensure_ascii=False)
    return hashlib.sha256(canonical.encode()).hexdigest()


print("Processing functions defined.")


In [ ]:
# Cell 8: Process Test Concepts Sheets

print(f"\n{'='*60}")
print("Processing test concepts sheets...")
print(f"{'='*60}")

all_results = []

# Get sheet names to process
sheets_to_process = [row['sheet_name'] for row in df_sheets.collect()] if sheets_count > 0 else []

for sheet_name in sheets_to_process:
    print(f"\nProcessing sheet: {sheet_name}")
    print("-" * 40)
    
    try:
        # Find header row using transfer variables
        header_row, tv_map, fixed_cols = find_header_row_by_transfer_vars(
            excel_bytes=excel_bytes,
            sheet_name=sheet_name,
            transfer_variable_names=transfer_variable_names
        )
        
        if header_row is None:
            print(f"  Skipping sheet - no valid header found")
            continue
        
        # Read the sheet with detected header
        raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, dtype=str)
        
        # Process data rows (after header)
        data_rows_processed = 0
        
        for row_idx in range(header_row + 1, len(raw_df)):
            row_values = raw_df.iloc[row_idx].values
            
            # Skip empty rows
            non_empty = [v for v in row_values if pd.notna(v) and str(v).strip()]
            if len(non_empty) < 2:
                continue
            
            # Build transfer_tuple_map
            transfer_tuple_map = {}
            
            # Add transfer variable columns
            for col_idx, tv_name in tv_map.items():
                val = row_values[col_idx] if col_idx < len(row_values) else None
                transfer_tuple_map[tv_name] = "" if pd.isna(val) else str(val).strip()
            
            # Add fixed columns
            for col_idx, col_name in fixed_cols.items():
                val = row_values[col_idx] if col_idx < len(row_values) else None
                transfer_tuple_map[col_name] = "" if pd.isna(val) else str(val).strip()
            
            # Extract test_concept_reference
            test_concept_ref = transfer_tuple_map.get('TEST_CONCEPT_REFERENCE', '')
            
            # Skip if no test concept reference
            if not test_concept_ref:
                continue
            
            # Compute definition hash
            definition_hash = compute_definition_hash(transfer_tuple_map)
            
            # Create record
            record = {
                'test_concept_id': str(uuid.uuid4()),
                'parent_document_id': PARENT_DOCUMENT_ID,
                'dta_id': None,
                'trial_id': TRIAL_ID,
                'data_stream_type': DATA_STREAM_TYPE,
                'data_provider_name': DATA_PROVIDER_NAME,
                'test_concept_reference': test_concept_ref,
                'transfer_tuple_map': transfer_tuple_map,
                'codelist_values': None,
                'variable_description': None,
                'definition_hash': definition_hash,
                'notes': f"Extracted from sheet: {sheet_name}, row: {row_idx + 1}",
                'status': 'COMPLETED',
                'vendor_comment': None,
                'version': None,
                'version_status': 'DRAFT',
                'is_current_draft': True
            }
            
            all_results.append(record)
            data_rows_processed += 1
        
        print(f"  ✓ Processed {data_rows_processed} data rows")
        
    except Exception as e:
        print(f"  ✗ Error processing sheet: {e}")
        import traceback
        traceback.print_exc()

print(f"\n{'='*60}")
print(f"Total test concepts extracted: {len(all_results)}")
print(f"{'='*60}")


In [ ]:
# DEBUG CELL: Check what's in the Excel after the header row
# Uncomment and run if you need to debug the sheet structure

# debug_sheet = "Test Concepts"  # Change to your sheet name
# header_row_debug = 9  # Change to detected header row

# raw_df_debug = pd.read_excel(BytesIO(excel_bytes), sheet_name=debug_sheet, header=None, dtype=str)

# print(f"\n=== DEBUG: Row {header_row_debug} (Header Row) ===")
# print(raw_df_debug.iloc[header_row_debug].values[:15])  # First 15 columns

# print(f"\n=== DEBUG: Row {header_row_debug + 1} (First Data Row) ===")
# if header_row_debug + 1 < len(raw_df_debug):
#     print(raw_df_debug.iloc[header_row_debug + 1].values[:15])

# print(f"\n=== DEBUG: Row {header_row_debug + 2} (Second Data Row) ===")
# if header_row_debug + 2 < len(raw_df_debug):
#     print(raw_df_debug.iloc[header_row_debug + 2].values[:15])

# print(f"\n=== DEBUG: Total rows in sheet: {len(raw_df_debug)} ===")
# print(f"=== DEBUG: Data rows after header: {len(raw_df_debug) - header_row_debug - 1} ===")


In [ ]:
# Cell 9: Preview Results

if all_results:
    print("\nSample Test Concept Records:")
    print("=" * 80)
    
    for i, record in enumerate(all_results[:5]):
        print(f"\n--- Record {i+1} ---")
        print(f"  test_concept_id: {record['test_concept_id'][:8]}...")
        print(f"  test_concept_reference: {record['test_concept_reference']}")
        print(f"  definition_hash: {record['definition_hash'][:16]}...")
        print(f"  transfer_tuple_map keys ({len(record['transfer_tuple_map'])}): {list(record['transfer_tuple_map'].keys())[:10]}...")
        print(f"  transfer_tuple_map sample:")
        for k, v in list(record['transfer_tuple_map'].items())[:5]:
            print(f"    {k}: {v[:50] if len(str(v)) > 50 else v}")
else:
    print("\n⚠️ No test concepts extracted. Check the sheet structure.")


In [ ]:
# Cell 10: Define Output Schema

test_concepts_schema = StructType([
    StructField("test_concept_id", StringType(), False),
    StructField("parent_document_id", StringType(), False),
    StructField("dta_id", StringType(), True),
    StructField("trial_id", StringType(), False),
    StructField("data_stream_type", StringType(), False),
    StructField("data_provider_name", StringType(), False),
    StructField("test_concept_reference", StringType(), False),
    StructField("transfer_tuple_map", MapType(StringType(), StringType()), False),
    StructField("codelist_values", ArrayType(StringType()), True),
    StructField("variable_description", StringType(), True),
    StructField("definition_hash", StringType(), True),
    StructField("notes", StringType(), True),
    StructField("status", StringType(), True),
    StructField("vendor_comment", StringType(), True),
    StructField("version", StringType(), True),
    StructField("version_status", StringType(), True),
    StructField("is_current_draft", BooleanType(), True)
])

print(f"Schema defined with {len(test_concepts_schema.fields)} fields")


In [ ]:
# Cell 11: Create DataFrame and Preview

if all_results:
    df_results = spark.createDataFrame(all_results, schema=test_concepts_schema)
    
    print(f"\nDataFrame created with {df_results.count()} rows")
    print("\nSchema:")
    df_results.printSchema()
    
    print("\nSample data:")
    df_results.select(
        "test_concept_id",
        "test_concept_reference",
        "transfer_tuple_map",
        "definition_hash"
    ).show(5, truncate=50)
else:
    print("No results to create DataFrame")


In [ ]:
# Cell 12: Write to Delta Table (Optional - set WRITE_TO_TABLE = True to enable)

WRITE_TO_TABLE = False  # Set to True to actually write

if WRITE_TO_TABLE and all_results:
    print(f"\n{'='*60}")
    print(f"Writing to {output_table}...")
    print(f"{'='*60}")
    
    # Add audit columns
    df_with_audit = df_results \
        .withColumn("created_by_principal", F.lit("test_notebook")) \
        .withColumn("created_ts", F.current_timestamp()) \
        .withColumn("last_updated_by_principal", F.lit("test_notebook")) \
        .withColumn("last_updated_ts", F.current_timestamp())
    
    # Write to table
    df_with_audit.write.format("delta") \
        .mode("append") \
        .option("mergeSchema", "true") \
        .saveAsTable(output_table)
    
    print(f"✓ Wrote {len(all_results)} records to {output_table}")
else:
    print(f"\n⚠️ WRITE_TO_TABLE is False - data NOT written to {output_table}")
    print("Set WRITE_TO_TABLE = True in Cell 12 to write data.")


In [ ]:
# Cell 13: Summary

print(f"\n{'='*80}")
print("SUMMARY")
print(f"{'='*80}")

print(f"""
Document: {PARENT_DOCUMENT_ID}
Trial: {TRIAL_ID}
Data Stream: {DATA_STREAM_TYPE}
Provider: {DATA_PROVIDER_NAME}

Transfer Variables Found: {transfer_vars_count}
Test Concepts Sheets: {sheets_count}
Test Concepts Extracted: {len(all_results)}

Output Table: {output_table}
Data Written: {'Yes' if WRITE_TO_TABLE else 'No (preview only)'}
""")

print("\nTest completed!")
